In [2]:
from deepagents import create_deep_agent
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import MemorySaver

In [3]:
from typing import Literal
from tavily import TavilyClient
from langchain_core.tools import tool
from dotenv import load_dotenv
load_dotenv()

tavily_client = TavilyClient()
@tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """
    Search the internet for information.
    
    Args:
        query: The search query string
        max_results: The maximum number of search results to return

        
    Returns:
        Search results as a dict
    """
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

In [4]:
llm = ChatOllama(
    model="gpt-oss:120b-cloud", # Use the model you pulled
    base_url="http://localhost:11434", # Point to the local Ollama server
    temperature=0
)

In [5]:
prompt = open("../promt.md","r",encoding="utf8").read()
agent_graph = create_deep_agent(
    model=llm,
    tools=[internet_search],
    system_prompt=prompt,
    checkpointer=MemorySaver() # This enables Short-Term Chat Memory
)

In [6]:
config = {"configurable": {"thread_id": "session_1"}}

In [7]:
events = agent_graph.stream(
    {"messages": [("user", "who is pm of india?")]},
    config,
    stream_mode="values"
)

In [8]:
final_answer = ""
for event in events:
    if "messages" in event:
        last_msg = event["messages"][-1]
        # Filter out intermediate tool calls, show only AI response
        if last_msg.type == "ai" and not last_msg.tool_calls:
            final_answer = last_msg.content

In [9]:
print(final_answer)


Narendra Modi is the current Prime Minister of India. (https://en.wikipedia.org/wiki/Narendra_Modi)


In [10]:
for i in agent_graph.checkpointer.list(config):
    print(i[4],"\n")

[] 

[('2174d610-307b-01d0-ffdf-b5295d33bf77', '__no_writes__', None)] 

[('32deea7d-1dc6-1e97-169c-b26e75f7657f', 'messages', [AIMessage(content='Narendra\u202fModi is the current Prime\u202fMinister of India. (https://en.wikipedia.org/wiki/Narendra_Modi)', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2026-02-07T03:16:49.949137997Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2289314553, 'load_duration': None, 'prompt_eval_count': 25994, 'prompt_eval_duration': None, 'eval_count': 133, 'eval_duration': None, 'logprobs': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--019c361a-154c-7941-9f5d-c9f5865b209a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 25994, 'output_tokens': 133, 'total_tokens': 26127})]), ('32deea7d-1dc6-1e97-169c-b26e75f7657f', 'branch:to:TodoListMiddleware.after_model', None)] 

[('2281a79e-37a1-d832-c5e5-4157af741f35', 'branch:to:model', None)] 

[('9e